In [ ]:
# this is example of Human in the loop, before executing decision, will ask for user confirmation
# not working some issue in getting user input and process further
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ.get("OPEN_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
#llm creation
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

#llm.invoke("who is the king in then north?")

In [ ]:
from langchain_core.tools import tool
from langgraph.types import interrupt, Command
@tool
def get_stock_price(stok: str)->str:
    ''' Return the current stock price of the given stock
    param stock: stock name
    return current price of the stock
    '''
    return {
        "HCL": 1280,
        "TCS": 3000,
        "IDFC": 200
    }.get(stok, 0)
    
@tool
def buy_stock_price(stock: str, quantity: int, total_price: float) -> str:
    '''
    Buy stock given the stock and quantity.
    Will ask user for approval.
    '''
    decision = interrupt(f"Approve buying {quantity} of {stock} for ₹{total_price}? (yes/no)")
    if decision.lower() == 'yes':
        return f"✅ Purchased {quantity} of {stock}."
    else:
        return "❌ Purchase cancelled."
    
tool_list = [get_stock_price,buy_stock_price]

#bind with llm tools
llm_with_tools = llm.bind_tools(tool_list)

#llm_with_tools.invoke("what is the price of HCL stock")

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

#state of the graph
class State(TypedDict):
	messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    # Preserve message history by appending the new response to the existing messages
    response = { "messages" : [llm_with_tools.invoke(state["messages"])]}
    return response

#graph configuation
memory = MemorySaver()
builder = StateGraph(State)
builder.add_node("chat_node", chatbot)
builder.add_node("tools", ToolNode(tool_list))
builder.add_edge(START, "chat_node")
builder.add_conditional_edges('chat_node', tools_condition)
builder.add_edge("tools", "chat_node")
graph = builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

#display the created graph as an image
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain.schema import HumanMessage
from pprint import pprint
# session id configuration we can define session id for our each thread
config = {'configurable': {'thread_id': 123}}
while True:
    input_msg = input("You: ")
    if input_msg in {"quit", "exit"}:
        break
    graph_state = graph.invoke({
        "messages": HumanMessage(content=input_msg)
    },config)
    last_msg = graph_state["messages"][-1]
    if graph_state.get("__interrupt__") :
        print('waiting for user input')
        print(last_msg)
        user_answer=  input("approve yes/no")
        response = graph.invoke(Command(resume=user_answer),config)
        
    pprint(last_msg)